# **Lane Regresor - Finding Lane Lines on the Road** 
***
This project uses the tools presented in the first lesson to identify lane lines on the road. First a pipeline of visual filters is used to identify edges on input images, then a set of line segments is extracted by applying the Hough Transform, and finally the position of the lane lines is derived from the detected line segments.

---

We start by importing general use packages and defining some basic visual filter functions:

In [10]:
import numpy as np
import cv2


def grayscale(image):
    r'''Converts the given BGR image to grayscale.
    '''
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def canny(grays, low_threshold, high_threshold):
    r'''Applies the Canny transform (controlled by the given arguments)
        to the given grayscale image, returning an edge map as result.
    '''
    return cv2.Canny(grays, low_threshold, high_threshold)


def gaussian_blur(image, kernel_size):
    r'''Applies a Gaussian smoothing transform of given kernel size to the image. 
    '''
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)


def region_of_interest(image, *vertices):
    r'''Return a transformed copy of the input image, where everything outside
        the polygon delimited by the given vertice list is blacked out.
    '''
    #defining a blank mask to start with
    mask = np.zeros_like(image)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, np.array([vertices]), ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image


def weighted_img(image, markers, α=0.8, β=1., λ=0.):
    r'''Merge the given input and marker images.
    '''
    return cv2.addWeighted(markers, α, image, β, λ)

Next we define classes for extracting and managing lines and line segments:

In [14]:
from collections import namedtuple
from sys import maxsize


class Line(list):
    r'''A line in 2D space, defined in terms of a slope `m` and an offset `b` such that `y = m * x + b`.
    '''
    def __init__(self, x1, y1, x2, y2):
        r'''Create a new line that passes through given points `(x1, y1)` and `(x2, y2)`.
        '''
        list.__init__(self)
        dx = x2 - x1
        dy = y2 - y1
        m = (dy / dx) if dx != 0 else maxsize
        b = y2 - x2 * m
        self.extend([m, b])

    def __call__(self, x):
        r'''Compute `y` such that `y = m * x + b` rounded down to integer precision.
        '''
        (m, b) = self
        return int(x * m + b)

    def draw(self, img, x1, x2, color=[255, 0, 0], thickness=2):
        r'''Draw this line on the given image, according to given arguments.
        '''
        y1 = self(x1)
        y2 = self(x2)
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)

    def update(self, line, λ=0.98):
        r'''Update this line's parameters according to the given arguments.
        '''
        (m, b) = self
        (m_i, b_i) = line
        self[0] = λ * m + (1 - λ) * m_i
        self[1] = λ * b + (1 - λ) * b_i
    
    @property
    def m(self):
        return self[0]
    
    @property
    def b(self):
        return self[1]

        
_Segment = namedtuple('_Segment', ['x1', 'y1', 'x2', 'y2'])

class Segment(_Segment):
    r'''A line segment delimited by two points `(x1, y1)` and `(x2, y2)`.
    '''
    def __new__(cls, x1, y1, x2, y2):
        r'''Create a new line segment.
        '''
        return _Segment.__new__(cls, x1, y1, x2, y2)

    @property
    def slope(self):
        (x1, y1, x2, y2) = self
        dx = x2 - x1
        dy = y2 - y1
        return (dy / dx) if dx != 0 else maxsize
        

class Segments(list):
    r'''A list of line segments.
    '''
    def __init__(self, *items):
        r'''Create a new list of line segments.
        '''
        list.__init__(self)
        for item in items:
            self.append(item)

    def dataset(self):
        r'''Convert this list of segments into a pair of matrices `(X, y)` suitable for input into
            regression methods.
            
            Each line segment `(x1, y1, x2, y2)` is converted to a sequence of `x2 - x1 + 1` points
            along the line that crosses `(x1, y1)` and `(x2, y2)`.
        '''
        (X, y) = ([], [])
        for segment in self:
            (x1, y1, x2, y2) = segment
            line = Line(x1, y1, x2, y2)
            for x in range(x1, x2 + 1):
                X.append(x)
                y.append(line(x))

        return (np.array([X]).T, np.array(y))

    def select(self, criterion):
        r'''Returns a new segment list containing only the elements that match the given criterion.
        '''
        selected = [segment for segment in self if criterion(segment)]
        return Segments(*selected)


class HoughSegments(Segments):
    r'''List of segments filled from the output of a Hough Transform.
    '''
    def __init__(self, img, rho, theta, threshold, min_line_len, max_line_gap):
        r'''Create a list of segments following edges on the input image.

            Input should be a line image, such as the output of a Canny transform.

            Lines are found by computing the Hough transform of the input image, then pairing down
            results according to the given arguments.
        '''
        Segments.__init__(self)
        for segments in cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), min_line_len, max_line_gap):
            for segment in segments:
                self.append(Segment(*segment))

Then we implement the Lane Regressor, a simple processing pipeline for identifying lane lines in video streams. It works by performing the following steps for each frame in a record:

1. Convert the frame to grayscale;
2. Apply Gaussian smoothing;
3. Apply the Canny transform to extract edges;
4. Cut out a triangular region with vertices on the image center and lower corners;
5. Use the Hough Transform to identify a set of line segments in the cut-out region;
6. Separate the identified segments in two sets with negative and positive slope;
7. Use RANSAC to derive an optimal representative segment for each set;
8. Use the RANSAC-derived representatives to update (or initialize) running estimates for the position of lane lines.

Steps 1-4 prepare the image frame for input to the Hough Transform line segment finder, with the objective of removing information not related to lane lines. It is expected that the output of step 5 will be a collection of line segments mostly clustered around the two lane lines. These are then separated in two sets based on slope signal: segments of negative slope are expected to be clustering on the left lane line, and positive slopes, on the right. This is a consequence of lane lines always looking somewhat slanted, owing to the way parallel lines drawn on the ground appear to converge as they reach for the horizon.

The two line segment sets are transformed into sets of points, which are used to train separate RANSAC regressors. RANSAC was chosen due to its ability to work well with data containing a clear trend but polluted with a relatively small number of outliers, as is the case here. The fitted lines are then used to update (or initialize) two running estimates of the position parameters of lane lines, which are drawn to the output frame. These estimates implement a model of gradual change over time, reflecting the way lane lines move slowly across the visual field in input videos.

In [54]:
from sklearn import linear_model


def filter_pipeline(image):
    r'''Pipeline of visual filters used to preprocess input images prior to line detection.
    '''
    (y_n, x_n) = image.shape[:2]
    x_c = x_n // 2
    x_l = x_n - 1
    y_c = y_n // 2
    y_l = y_n - 1

    grays = grayscale(image)
    grays = gaussian_blur(grays, 5)
    grays = canny(grays, 50, 150)
    grays = region_of_interest(grays, (0, y_l), (x_c, y_c), (x_l, y_l))
    
    return grays


class LaneRegressor(object):
    r'''Lane line position online regressor.
    '''
    def __init__(self):
        r'''Create a new regressor.
        '''
        self.lines = dict()
    
    def __call__(self, image):
        r'''Process the given image frame.
        '''
        segments = HoughSegments(filter_pipeline(image), 1, np.pi / 180.0, 10, 10, 2)
        
        (y_n, x_n) = image.shape[:2]
        x_c = x_n // 2
        x_l = x_n - 1
        m = y_n / x_n
        
        self.update('l', segments, lambda s: s.slope < -m)
        self.update('r', segments, lambda s: m < s.slope < maxsize)
        
        lines_img = np.zeros(image.shape, dtype=image.dtype)
        self.draw('l', lines_img, 0, x_c)
        self.draw('r', lines_img, x_c, x_l)

        return weighted_img(lines_img, image)

    def draw(self, key, image, x1, x2):
        r'''Draw the named line on the image, if found. Otherwise does nothing.
        '''
        line = self.lines.get(key)
        if line == None:
            return
        
        line.draw(image, x1, x2, thickness=7)
    
    def update(self, key, segments, criterion):
        r'''Update (or initialize) the running estimate for the named line, based on data
            extracted from the given segment list.
        '''
        selected = segments.select(criterion)
        if len(selected) == 0:
            return
        
        (X, y) = selected.dataset()
        model = linear_model.RANSACRegressor(linear_model.LinearRegression())
        model.fit(X, y)
        
        (x1, x2) = (X.min(), X.max())
        y1 = int(model.predict(x1)[0])
        y2 = int(model.predict(x2)[0])
        updating = Line(x1, y1, x2, y2)

        line = self.lines.get(key)
        if line == None:
            self.lines[key] = updating
        else:
            line.update(updating)

Before starting tests, let's define a few conveniences for processing and diplaying videos:

In [67]:
from random import random
from moviepy.editor import VideoFileClip
from IPython.display import HTML


# Template HTML snippet for embedding a video display into the notebook.
VIDEO_TAG = r'''
<video width="960" height="540" controls>
  <source src="%s?t=%f">
</video>
'''


def process(path_in, path_out):
    r'''Process the video at the input path, saving the result to the output video file path.
    '''
    clip1 = VideoFileClip(path_in)
    white_clip = clip1.fl_image(LaneRegressor())
    # Display processing progress inline.
    %time white_clip.write_videofile(path_out, audio=False)

Now with all pieces in place, let's start by testing the regressor on the first example video:

In [68]:
process('solidWhiteRight.mp4', 'white.mp4')

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:10<00:00, 20.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 1min 4s, sys: 2.36 s, total: 1min 7s
Wall time: 10.6 s


Results can be visualized inline using the snippet below:

In [69]:
HTML(VIDEO_TAG % ('white.mp4', random())) # the second, random parameter forces the browser to reload the video.

Now for the one with the solid yellow lane on the left:

In [57]:
process('solidYellowLeft.mp4', 'yellow.mp4')

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:36<00:00, 17.07it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 3min 36s, sys: 9.1 s, total: 3min 46s
Wall time: 36.6 s


In [58]:
HTML(VIDEO_TAG % ('yellow.mp4', random()))

And last for the challenge video:

In [65]:
process('challenge.mp4', 'extra.mp4')

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:22<00:00, 11.49it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 2min 13s, sys: 5.5 s, total: 2min 19s
Wall time: 23.5 s


In [66]:
HTML(VIDEO_TAG % ('extra.mp4', random()))

## Reflections

The Lane Regressor is a simple processing pipeline for identifying lane lines in video streams. It combines visual processing and regression techniques to iteratively compute an estimate for two image regions within which the left and right lane lines are expected to be. Experiments show it performs adequately on the example videos.

As proposed the system includes a number of parameters (such as those for the Canny and Hough Transforms) for which there's no clear optimization procedure, requiring values to be determined manually through trial-and-error; consequently there's no indication the settings used for the cases above will be optimal for differing inputs. Line identification is primitive, with slope being the only parameter to judge whether a segment should be discarded or kept for further processing; this could make the algorithm mistake some other object for a lane line, if it produces more segments of the right slope. The use of a random regressor (RANSAC) may cause similar or repeated inputs to produce noticeably different outputs, which may lead to consistency issues. The running estimate model assumes the current position of lane lines in the visual field varies smoothly over time, but in principle there's no guarantee that will always be the case.

Addressing the above issues would go a long way towards improving system performance. Still, the general procedure outlined at this point &ndash; a filter pipeline to extract relevant visual information, followed by extraction of model instances (line segments in this case) that are used to generate input data to an online learning system &ndash; showed promise, deserving further investigation.